In [ ]:
# pip install simpy

Note: you may need to restart the kernel to use updated packages.


In [5]:
import simpy
import pandas as pd
import random

# Cargar dataset
df = pd.read_csv('rssi_2018_05.csv')

class WiFiSimulation:
    def __init__(self, env, df):
        self.env = env
        self.df = df
        self.log = []

    def get_random_block_data(self, mac_cliente):
        """Sorteo con reposición de un bloque para un cliente"""
        blocks = self.df[self.df['mac_cliente'] == mac_cliente]['block_index'].unique()
        chosen_block = random.choice(blocks)
        return self.df[(self.df['mac_cliente'] == mac_cliente) & 
                       (self.df['block_index'] == chosen_block)]

    def cliente_process(self, mac_id, t_arribo, duracion):
        # 1. Esperar al tiempo de arribo sorteado
        yield self.env.timeout(t_arribo)
        
        # 2. Conexión Inicial (t*)
        estado_actual = self.get_random_block_data(mac_id)
        # Buscamos el mejor AP en la foto inicial
        mejor_ap_inicial = estado_actual.loc[estado_actual['rssi'].idxmax()]
        ap_actual = mejor_ap_inicial['mac_ap']
        rssi_actual = mejor_ap_inicial['rssi']
        
        self.registrar_evento(mac_id, ap_actual, rssi_actual, "Conexión Inicial")

        # 3. Bucle de permanencia (t*+1 ...)
        for _ in range(duracion - 1):
            yield self.env.timeout(1)
            
            # Sorteo de nueva foto (nuevo block_index)
            estado_actual = self.get_random_block_data(mac_id)
            
            # Verificamos cómo se ve nuestro AP actual en la nueva foto
            row_ap_actual = estado_actual[estado_actual['mac_ap'] == ap_actual]
            
            # Si el AP actual no aparece en la nueva foto, asumimos señal nula
            rssi_actual = row_ap_actual['rssi'].iloc[0] if not row_ap_actual.empty else -100
            
            # Buscamos cuál sería el mejor AP en esta nueva foto por si hay que saltar
            mejor_opcion = estado_actual.loc[estado_actual['rssi'].idxmax()]
            mejor_rssi_posible = mejor_opcion['rssi']
            
            # Lógica Sticky
            hacer_handover = False
            causa = ""
            
            if rssi_actual < -85:
                hacer_handover = True
                causa = "Señal Crítica"
            elif mejor_rssi_posible > -50 and ap_actual != mejor_opcion['mac_ap']:
                hacer_handover = True
                causa = "Mejor Opción (> -50)"
            
            if hacer_handover:
                ap_actual = mejor_opcion['mac_ap']
                rssi_actual = mejor_rssi_posible
                self.registrar_evento(mac_id, ap_actual, rssi_actual, f"Handover ({causa})")
            else:
                self.registrar_evento(mac_id, ap_actual, rssi_actual, "Mantiene (Sticky)")

    def registrar_evento(self, mac, ap, rssi, evento):
        self.log.append({
            'tiempo': self.env.now,
            'cliente': mac,
            'ap': ap,
            'rssi': rssi,
            'evento': evento
        })

# --- Ejecución ---
env = simpy.Environment()
sim = WiFiSimulation(env, df)

# Ejemplo: Sorteamos que el cliente X llega en t=5 y dura 10 unidades
mac_ejemplo = "0.135.1.143.215.55" 
env.process(sim.cliente_process(mac_ejemplo, t_arribo=5, duracion=10))

env.run()

# Resultados
resultados_df = pd.DataFrame(sim.log)
print(resultados_df)

   tiempo             cliente                     ap  rssi  \
0       5  0.135.1.143.215.55  44.208.45.127.103.160   -68   
1       6  0.135.1.143.215.55  44.208.45.127.103.160   -72   
2       7  0.135.1.143.215.55   44.208.45.106.167.16   -83   
3       8  0.135.1.143.215.55   44.208.45.106.167.16   -83   
4       9  0.135.1.143.215.55   44.208.45.106.167.16   -81   
5      10  0.135.1.143.215.55   44.208.45.106.167.16   -83   
6      11  0.135.1.143.215.55   44.208.45.106.167.16   -81   
7      12  0.135.1.143.215.55   44.208.45.106.167.16   -84   
8      13  0.135.1.143.215.55   44.208.45.106.167.16   -81   
9      14  0.135.1.143.215.55  44.208.45.127.103.160   -68   

                     evento  
0          Conexión Inicial  
1         Mantiene (Sticky)  
2  Handover (Señal Crítica)  
3         Mantiene (Sticky)  
4         Mantiene (Sticky)  
5         Mantiene (Sticky)  
6         Mantiene (Sticky)  
7         Mantiene (Sticky)  
8         Mantiene (Sticky)  
9  Handover (Seña